In [6]:
# train on 1000 actual pages randomly from wikipedia (create the training set)
import re
import requests
import wikipedia
import nltk
import random
import pandas as pd
from tqdm import tqdm
from ratelimit import limits, sleep_and_retry
def gather_page(category, page_dict):
    # Get a random page from wikipedia in the category
    page = wikipedia.random(category=category)
    # Get the page content
    content = wikipedia.page(page)
    text_content = content.content
    related = content.links
    # Add the content to the dictionary
    page_dict["title"].append(page) # append the title
    page_dict["text"].append(text_content) # append the text content
    page_dict["related"].append(related) # supplied at training time
    page_dict["category"].append(category) # supplied at training time
    return page_dict # return the page dictionary

def gather_training_data(pages_count, category):
    # Gather the pages from wikipedia
    training_set = [] # list of dictionaries with title, text, and category
    page_dict = {"title": [], "text": [], "related":[], "category": []}
    for i in tqdm(range(pages_count)):
        page_dict = gather_page(category, page_dict) # gather the page (1) and add it to the training set (2)
        print("Gathered page " + str(i+1) + " of " + str(pages_count) + " for category " + category + ".")
        print("Page title: " + page_dict["title"][-1]) # print the title of the page
        training_set.append(page_dict) # add the page to the training set
        # add the training set to the training set

    # save the data to a csv file for later use
    df = pd.DataFrame(page_dict) # convert to a dataframe for easy saving
    filename = "./data/training_data_" + category + ".csv"
    df.to_csv(filename, index=False) # save the data to a csv file
    return training_set


categories = ["Living people", "Popular culture", "Internet memes", "Viral videos", "Social media","Social_phenomena","Phenomena","Pseudohistory","Hoaxes","Hoaxes_in_science","Written_fiction_presented_as_fact","Memetics","TikTokers","Social_media","Fake_news","Word_play","IOS_games","IOS_software","Works_involved_in_plagiarism_controversies","YouTube_controversies","2000s_YouTube_series","2010s_YouTube_series","Psychotherapy","Psychology","Human medicine","Interpersonal_relationships","Psychological_therapy","Human_behavior","Love","Philosophers_of_ethics_and_morality","United_States_Supreme_Court_cases","Political_party_founders","Classics_educators","Fictional_inventors","Whistleblowing","News_leaks","WikiLeaks","Popular_music","Puzzles","Fiction_about_personifications_of_death","Bogeymen","Comedians","Authors","Artists","Literary_concepts","Historical_eras","Viral_videos","Internet_memes","Theorems","21st-century_male_actors","21st-century_female_actors","Fables","American_Internet_celebrities","Legends","Mythology","Rules_of_thumb","Adages","Fallacies","Mountains","Lakes","Oceans","Sea_Monsters","fairy_tales","1800s","Tall_tales","Urban_legends","Superstitions","Western_culture","Quotations_from_film","Quotations_from_music","Quotations_from_literature","Quotations_from_television","Quotations_from_video_games","English_phrases", "English_proverbs", "English-language_slang", "English_slang","Universal_Pictures_films", "Paramount_Pictures_films","Human Anatomy","Human Physiology","Human Behavior","Melons","Fruit","Vegetables","Flowers","Trees","Dinosaurs_in_popular_culture","Extraterrestrial_life_in_popular_culture",'English-language_idioms','Catchphrases','British_English_idioms','Living_people',"Philosophers_of_ethics_and_morality","United_States_Supreme_Court_cases","Political_party_founders","Classics_educators","Social_phenomena","Phenomena","Pseudohistory","Hoaxes","Hoaxes_in_science","Written_fiction_presented_as_fact","Viral_videos"]


In [7]:
import requests
import time # for rate limiting
from tqdm import tqdm

# Replace CATEGORY with the name of the category you want to search
CATEGORY = "Living people"

def request_pages(CATEGORY):

    # Make a request to the Wikipedia API to search for pages in the specified category
    response = requests.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "format": "json",
        "list": "categorymembers",
        "cmtitle": f"Category:{CATEGORY}",
        "cmlimit": "max" # Set the maximum number of pages to return
    })

    page_dict = {"title": [], "text": [], "related":[], "category": []}
    
    # Get the list of pages from the response
    pages = response.json()["query"]["categorymembers"]

    # get the content of each page
    for page in tqdm(pages):
        try:
            # Get the title of the page
            title = page["title"]
            # Get the content of the page
            page = wikipedia.page(title)
            # Add the page to the page dictionary
            page_dict["title"].append(title)
            page_dict["text"].append(page.content)
            page_dict["related"].append(page.links) # supplied at training time
            page_dict["category"].append(CATEGORY)
            # every ten pages, save the data to a csv file
            if len(page_dict["title"]) % 10 == 0:
                # convert the dict to a dataframe
                df = pd.DataFrame.from_dict(page_dict, orient='index', columns=['summary'])
                # save the dataframe to a csv file
                df.to_csv("./data/summary_data_" + CATEGORY + ".csv")
            time.sleep(0.75) # rate limit the requests
        except:
            pass
    return page_dict


In [8]:
# call the function
page_dict = request_pages(CATEGORY)

  8%|▊         | 42/500 [00:50<06:28,  1.18it/s]/opt/anaconda3/envs/groupme/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/anaconda3/envs/groupme/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
 11%|█         | 56/500 [01:07<09:49,  1.33s/it]/opt/anaconda3/envs/groupme/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually i